<a href="https://colab.research.google.com/github/mekhi-woods/HiloCATsSN1991bg/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [452]:
# """ DEPRICATED FUNCTIONS """
# def plot_91bglike_SN1a(FILTER_WHEEL = ['u', 'g', 'r', 'i', 'B', 'V0']):
    # KrisciunasPath = "/content/HiloCATsSN1991bg/targetLists/91bglike_justnames.txt"
    # KrisciunasNames = np.genfromtxt(KrisciunasPath, dtype=str, delimiter=', ')

    # allCPSPhot = "/content/HiloCATsSN1991bg/data/CSPdata/SN_photo.dat"
    # allCPSPhotData = np.genfromtxt(allCPSPhot, dtype='str')

    # names = allCPSPhotData[:,0]
    # filters = allCPSPhotData[:,1]
    # time = allCPSPhotData[:,2]
    # light = allCPSPhotData[:,3]
    # err = allCPSPhotData[:,4]

    # plt.figure(figsize=(10,6))
    # sigma = 1
    # for tar in KrisciunasNames:
    #     for n in range(len(FILTER_WHEEL)):
    #         # output_names = names[(names == tar) & (filters == FILTER_WHEEL[n])]
    #         output_light = light[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64')
    #         output_time = time[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64') + 53000
    #         output_err = err[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64')
    #         plt.errorbar(output_time, output_light, yerr=output_err*sigma, fmt='o', label=FILTER_WHEEL[n])

    #     plt.title(tar); plt.xlabel('Time [MJD]'); plt.ylabel('Intensity [mag]')
    #     plt.gca().invert_yaxis()
    #     plt.legend()
    #     # plt.savefig('save\\'+str(tar)+'.png')
    #     plt.show()
    #     break
    # return

# def plot_DR3_tmax_st_EBVhost():
    # data = np.genfromtxt('/content/HiloCATsSN1991bg/DR3_fits.dat', dtype=str, skip_header=1)
    # DR3_st = np.stack((data[:, 1].astype(float), data[:, 2].astype(float)), axis=1)
    # DR3_Tmax = np.stack((data[:, 5].astype(float), data[:, 6].astype(float)), axis=1)
    # DR3_EBVhost = np.stack((data[:, 25].astype(float), data[:, 26].astype(float)), axis=1)

    # fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,5))
    # fig.suptitle("DR3's Tmax vs. st vs EBVhost")
    # sigmas = [[1, 1], [1, 1], [1, 1]]

    # # Tmax vs. st
    # ax1.errorbar(DR3_Tmax[:, 0], DR3_st[:, 0],
    #              xerr=DR3_Tmax[:, 1]*sigmas[0][0], yerr=DR3_st[:, 1]*sigmas[0][1],
    #              fmt='yo')
    # ax1.set_xlabel('Tmax'); ax1.set_ylabel('st')
    # ax1.set_title('Tmax vs. st, sigma(x='+str(sigmas[0][0])+', y='+str(sigmas[0][1])+')')

    # # st vs. EBVhost
    # ax2.errorbar(DR3_st[:, 0], DR3_EBVhost[:, 0],
    #              xerr=DR3_st[:, 1]*sigmas[1][0], yerr=DR3_EBVhost[:, 1]*sigmas[1][1],
    #              fmt='bo')
    # ax2.set_xlabel('st'); ax2.set_ylabel('EBVhost')
    # ax2.set_title('st vs. EBVhost, sigma(x='+str(sigmas[1][0])+', y='+str(sigmas[1][1])+')')

    # # Tmax vs. EBVhost
    # ax3.errorbar(DR3_Tmax[:, 0], DR3_EBVhost[:, 0],
    #              xerr=DR3_Tmax[:, 1]*sigmas[2][0], yerr=DR3_EBVhost[:, 1]*sigmas[2][1],
    #              fmt='ro')
    # ax3.set_xlabel('Tmax'); ax3.set_ylabel('EBVhost')
    # ax3.set_title('Tmax vs. EBVhost, sigma(x='+str(sigmas[2][0])+', y='+str(sigmas[2][1])+')')

    # plt.show()
    # return



In [453]:
# """ START UP """
# import os
# import shutil
# if os.path.exists('/content/HiloCATsSN1991bg') == True:
#     shutil.rmtree('/content/HiloCATsSN1991bg')
#     !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git
# else:
#     !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git

# !pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple snpy
# !pip install requests


In [454]:
""" IMPORTS """
import os
import time
import glob
import json
import snpy
import shutil
import requests
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
from zipfile import ZipFile
from requests.auth import HTTPBasicAuth
from HiloCATsSN1991bg.scripts import tns_redshifts

In [455]:
""" GLOBALS """
# Paths
ROOT_SAVE = '/content/HiloCATsSN1991bg/saved_data/'
MODELS_PATH = ROOT_SAVE+'models/'
PLOTS_PATH = ROOT_SAVE+'plots/'
DATA_PATH = ROOT_SAVE+'data/'
ATLAS_PLOTS_PATH = PLOTS_PATH+'atlas/'
ATLAS_SNPY_PLOTS_PATH = PLOTS_PATH+'atlas_snpy/'
ATLAS_DATA_PATH = DATA_PATH+'atlas/'
ATLAS_SNPY_DATA_PATH = DATA_PATH+'atlas_snpy/'
ZTF_DATA_PATH = DATA_PATH+'ztf/'

API_TOKEN = "7f4e1dee8f52cf0c8cdaf55bf29d04bef4810fb4"
USE_PAST_CALC = True # Uses past calculations to speed up code
USE_SAVED = False

# Credintials
tns_bot_id = '73181'
tns_bot_name = 'YSE_Bot1'
tns_bot_api_key = '0d771345fa6b876a5bb99cd5042ab8b5ae91fc67'

In [456]:
""" CLASSES """
class ATLAS_SN():
    def __init__(self, ATLASname='EmptySN', objname='SN####abc', RA=0.00000000000, DEC=0.00000000000, z=0.00000000000,
                 DM=0.00, st=0.00, EBVhost=0.00, Tmax=0.00, mu=0.00,
                 t_o=np.array([0]), flux_o=np.array([0]), dflux_o=np.array([0]), mag_o=np.array([0]), dmag_o=np.array([0]),
                 t_c=np.array([0]), flux_c=np.array([0]), dflux_c=np.array([0]), mag_c=np.array([0]), dmag_c=np.array([0])):
        self.ATLASname=ATLASname
        self.objname=objname
        self.RA=RA
        self.DEC=DEC
        self.z=z

        self.t_o=t_o
        self.flux_o=flux_o
        self.dflux_o=dflux_o
        self.mag_o=mag_o
        self.dmag_o=dmag_o

        self.t_c=t_c
        self.flux_c=flux_c
        self.dflux_c=dflux_c
        self.mag_c=mag_c
        self.dmag_c=dmag_c

        self.mu = mu
        self.DM = DM
        self.st = st
        self.EBVhost = EBVhost
        self.Tmax = Tmax
        return

    def __str__(self):
        return (self.objname+' : '+self.ATLASname+' @ ('+str(self.RA)+', '+str(self.DEC)+')\n'+
                'O-Filter:'+
                '\t t [MJD]: '+str(np.min(self.t_o))+'...'+str(np.max(self.t_o))+'\n'+
                '\t\t flux [uJy]: '+str(np.min(self.flux_o))+'...'+str(np.max(self.flux_o))+'\n'+
                '\t\t dflux [duJy]: '+str(np.min(self.dflux_o))+'...'+str(np.max(self.dflux_o))+'\n'+
                '\t\t mag: '+str(np.min(self.mag_o))+'...'+str(np.max(self.mag_o))+'\n'+
                '\t\t dmag: '+str(np.min(self.dmag_o))+'...'+str(np.max(self.dmag_o))+'\n'+
                'C-Filter:'+
                '\t t [MJD]: '+str(np.min(self.t_c))+'...'+str(np.max(self.t_c))+'\n'+
                '\t\t flux [uJy]: '+str(np.min(self.flux_c))+'...'+str(np.max(self.flux_c))+'\n'+
                '\t\t dflux [duJy]: '+str(np.min(self.dflux_c))+'...'+str(np.max(self.dflux_c))+'\n'
                '\t\t mag: '+str(np.min(self.mag_c))+'...'+str(np.max(self.mag_c))+'\n'+
                '\t\t dmag: '+str(np.min(self.dmag_c))+'...'+str(np.max(self.dmag_c))+'\n'+
                'Params:'+
                '\t Distance Mod [mu]: '+str(self.mu)+'\n'+
                '\t\t DM: '+str(self.DM)+'\n'+
                '\t\t Shape [st]: '+str(self.st)+'\n'
                '\t\t EBVhost: '+str(self.EBVhost)+'\n'+
                '\t\t Peak Time [Tmax]: '+str(self.Tmax)+'\n')

    def coords(self):
        return('('+str(self.RA)+', '+str(self.DEC)+')')

    def flux_to_mag(self):
        return [[-2.5*np.log10(self.flux_o) + 23.9, 1.0857*self.dflux_o/self.flux_o],
                [-2.5*np.log10(self.flux_c) + 23.9, 1.0857*self.dflux_c/self.flux_c]]


In [457]:
""" FUNCTIONS """
# General functions
def recover_dir():
    if os.path.exists(ROOT_SAVE) == False:
        os.mkdir(ROOT_SAVE)
    if os.path.exists(MODELS_PATH) == False:
        os.mkdir(MODELS_PATH)
    if os.path.exists(PLOTS_PATH) == False:
        os.mkdir(PLOTS_PATH)
    if os.path.exists(DATA_PATH) == False:
        os.mkdir(DATA_PATH)
    if os.path.exists(ATLAS_PLOTS_PATH) == False:
        os.mkdir(ATLAS_PLOTS_PATH)
    if os.path.exists(ATLAS_SNPY_PLOTS_PATH) == False:
        os.mkdir(ATLAS_SNPY_PLOTS_PATH)
    if os.path.exists(ATLAS_DATA_PATH) == False:
        os.mkdir(ATLAS_DATA_PATH)
    if os.path.exists(ATLAS_SNPY_DATA_PATH) == False:
        os.mkdir(ATLAS_SNPY_DATA_PATH)
    if os.path.exists(ZTF_DATA_PATH) == False:
        os.mkdir(ZTF_DATA_PATH)
    if os.path.exists(ATLAS_SNPY_PLOTS_PATH+'/snpyplots.zip'):
        os.remove(ATLAS_SNPY_PLOTS_PATH+'/snpyplots.zip')
    return

def TNS_details(ra, dec):
    # Code from David

    headers = tns_redshifts.build_tns_header(tns_bot_id, tns_bot_name)
    tns_api_url = f"https://www.wis-tns.org/api/get"

    # get the API URLs
    search_tns_url = tns_redshifts.build_tns_url(tns_api_url, mode="search")
    get_tns_url = tns_redshifts.build_tns_url(tns_api_url, mode="get")

    search_data = tns_redshifts.build_tns_search_query_data(tns_bot_api_key, ra, dec)
    transients = tns_redshifts.rate_limit_query_tns(search_data, headers, search_tns_url)

    get_data = tns_redshifts.build_tns_get_query_data(tns_bot_api_key, transients[0])
    transient_detail = tns_redshifts.rate_limit_query_tns(get_data, headers, get_tns_url)

    return transient_detail

# Fitting functions
def snpy_fit(filePath, model='max_model', shapeParam='dm15', BandsToFit = ['B','g','r','i'], summarize=True, saveplots=False, saveLoc=ATLAS_SNPY_PLOTS_PATH, show_plots=True):
    s = snpy.get_sn(filePath)

    # Set model parameters
    s.choose_model(model, stype=shapeParam)
    s.set_restbands() # Auto pick appropriate rest-bands

    # Fit data -- using David configurations
    fitargs = {'mangle':1,'calibration':0, 'quiet':False} # I don't remember what calibration is
    s.fit(BandsToFit,
          dokcorr=True,
          k_stretch=False,
          reset_kcorrs=True,
          **fitargs)

    # Show results
    if summarize:
        s.summary()
        # for param in s.parameters:
        #     print("{} = {} +/- {}".format(param, s.parameters[param], s.errors[param]))
    if saveplots:
        print('Plot saved to', ATLAS_SNPY_PLOTS_PATH+filePath[-17:-9]+'_snpyplots.png')
        s.plot(outfile=ATLAS_SNPY_PLOTS_PATH+filePath[-17:-9]+'_snpyplots.png')
        if show_plots:
            plt.show()
        else:
            plt.close()

        with ZipFile(ATLAS_SNPY_PLOTS_PATH+'snpyplots.zip', 'a') as zip_object:
            zip_object.write(ATLAS_SNPY_PLOTS_PATH+filePath[-17:-9]+'_snpyplots.png')

    return s

def SNooPy2_fitting(dataPath, tarNames, model='EBV_model2', shape='st', bands=['B','g','r','i'], output=False, snpyPlots=False, show_plots=True, use_saved=True):
    problemChildren = []

    if len(bands) == 0:
        bands = None

    tarPaths = []
    for tar in tarNames:
        tarPaths.append(dataPath+str(tar))

    SNe_mu = {}
    SNe_z = {}
    SNe_params = {}

    for i in range(len(tarPaths)):
        tarName = tarNames[i][:-9]
        tarSave = MODELS_PATH+tarName+'_'+model+'.snpy'
        print('[ '+str(i+1)+' / '+str(len(tarPaths))+'] Fiting data for '+tarName+'...')

        # Create/Retrieve Fit
        valid = True

        if os.path.exists(tarSave) and use_saved:
            print(tarName, 'found in files! Pulling data...')
            s_n = snpy.get_sn(tarSave)
        elif os.path.exists(tarPaths[i]):
            try:
                s_n = snpy_fit(tarPaths[i],
                                model=model,
                                shapeParam=shape,
                                BandsToFit=None,
                                summarize=output,
                                saveplots=snpyPlots,
                                saveLoc=ATLAS_SNPY_PLOTS_PATH,
                                show_plots=show_plots) # Enter snpy fit function
            except ValueError:
                problemChildren.append(tarName)
                print('[!!!] ValueError: No data near maximum light... skipping\n')
                valid = False
            except RuntimeError:
                problemChildren.append(tarName)
                print('[!!!] RuntimeError: Model has trailed off fitting filter... skipping\n')
                valid = False
            except TypeError:
                problemChildren.append(tarName)
                print('[!!!] TypeError: m > k must hold (I have no clue what this means)... skipping\n')
                valid = False
            except:
                problemChildren.append(tarName)
                print('[!!!] Unknown Error... skipping\n')
                valid = False
        else:
            print(tarName, 'does not exsist in CSP data... skipping')
            valid = False

        if valid:
            # Save model
            s_n.save(tarSave)

            # Pull SNooPY distance
            snpy_mu = s_n.get_distmod() # Nab paramaters from SNe objects)

            # Update dictionary/list
            SNe_mu.update({tarName: snpy_mu})
            SNe_z.update({tarName: s_n.z})
            SNe_params.update({tarName: [s_n.st, s_n.Tmax, s_n.EBVhost, s_n.DM]})

            # Print info
            print('Redshift:\t z = '+str(s_n.z))
            print('Distance: \t mu = '+str(round(snpy_mu, 4))+'\n')

    print('Problem children:\n', '[', len(problemChildren), ']', problemChildren)
    plt.close()
    return SNe_mu, SNe_z, SNe_params

def ATLAS_snpy_fitting(SNe, ATLASnames, show_plot=True, use_saved=True):
    SNe_mu, SNe_z, SNe_params = SNooPy2_fitting(ATLAS_SNPY_DATA_PATH,
                                                ATLASnames,
                                                model='EBV_model2', shape='st', bands=['ATri', 'ATgr'],
                                                output=True, snpyPlots=True, show_plots=show_plot, use_saved=use_saved)
    SNe_new = []
    for SN in SNe:
        try:
            SN.mu = SNe_mu[SN.objname]
            SN.z = SNe_z[SN.objname]
            SN.st = SNe_params[SN.objname][0]
            SN.Tmax = SNe_params[SN.objname][1]
            SN.EBVhost = SNe_params[SN.objname][2]
            SN.DM = SNe_params[SN.objname][3]
            SNe_new.append(SN)
        except KeyError:
            print(SN.objname, 'is a problem child, removing...')

    # Save distance mod
    params = np.genfromtxt(ROOT_SAVE+'ATLAS_variables.txt', delimiter=', ', dtype=str, skip_header=1)
    with open(ROOT_SAVE+'ATLAS_variables_mu.txt', 'w') as f:
        f.write('objname, ATLASname, RA, DEC, z, tmin, tmax, mu\n')
        for n in range(len(SNe_new)):
            f.write(str(params[n, 0])+', '+str(params[n, 1])+', '+str(params[n, 2])+', '+str(params[n, 3])+', '+
                    str(params[n, 4])+', '+str(params[n, 5])+', '+str(params[n, 6])+', '+str(SNe_new[n].mu)+'\n')

    return SNe_new

# Plotting functions
def plot_SNooPy_mu_vs_z(save_plot=True, saveLoc=ATLAS_SNPY_PLOTS_PATH):
    data = np.genfromtxt(ROOT_SAVE+'ATLAS_variables.txt', delimiter=', ', dtype=str, skip_header=1)
    name, mu, z = data[:, 0], data[:, 7].astype(float), data[:, 4].astype(float)

    print(mu)
    print(z)


    for n in range(len(name)):
        plt.loglog(z[n], mu[n], 'o')
        if z[n] < 0.0125:
            plt.text(z[n], mu[n], name[n], fontsize='xx-small')
    plt.xlabel('Redshift [z]'); plt.ylabel('Distance Modulus [mu]')
    plt.title('ATLAS SNe -- Fitted with SNooPy \nDistance Modulus v. Redshift')
    if save_plot:
        plt.savefig(saveLoc+'SNooPy_mu_vs_z.png')
    plt.show()
    return

def plot_SNooPy_hist(binNum=10, save_plot=True, saveLoc=ATLAS_SNPY_PLOTS_PATH):
    data = np.genfromtxt(ROOT_SAVE+'ATLAS_variables.txt', delimiter=', ', dtype=str, skip_header=1)
    paramsData = [data[:, 3].astype(float), data[:, 4].astype(float), data[:, 5].astype(float)]
    paramsNames = ['st', 'ATLAS SNooPy Fitting Parameters \n Tmax', 'EBVhost']

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    for n in range(len(paramsNames)):
        axes[n].hist(paramsData[n], bins=binNum)
        axes[n].set_title(paramsNames[n])
    if save_plot:
        plt.savefig(saveLoc+'SNooPy_params_hist.png')
    plt.show()


    return

def plot_DvD(snpyDistances, burnsDistances, size=(8,5), save=False):
    print("Ploting differences in distance calculations...")
    plt.figure(figsize=size)
    for x in snpyDistances:
        try:
            plt.scatter(burnsDistances[x], snpyDistances[x], marker='o')
            plt.text(burnsDistances[x]+0.05, snpyDistances[x]-0.05, x, fontsize='xx-small')
            print('Burns Distance:', burnsDistances[x], '| SNooPy Distance:', snpyDistances[x])
        except KeyError:
            print(x, 'not found.')
            pass
    plt.title("Burns v. SNooPy Distance")
    plt.xlabel('Burns Distance'); plt.ylabel('SNooPy Distance')
    plt.xlim(min(burnsDistances.values()), max(burnsDistances.values()))
    plt.ylim(min(burnsDistances.values()), max(burnsDistances.values()))

    if save:
        plt.savefig('burns_v_snpy_dist.png')

    plt.show()
    return

def plot_residuals(snpyDistances, burnsDistances, snpyRedshifts, size=(8,5), save=False):
    print("Ploting residuals...")
    plt.figure(figsize=size)
    for x in snpyDistances:
        try:
            plt.scatter(snpyRedshifts[x], abs(burnsDistances[x]-snpyDistances[x]), marker='o')
            plt.text(snpyRedshifts[x], abs(burnsDistances[x]-snpyDistances[x]), x, fontsize='xx-small')
            print('Redshift:', snpyRedshifts[x], '| Burns-SNooPy Distance:', burnsDistances[x]-snpyDistances[x])
        except KeyError:
            print(x, 'not found.')
            pass
    plt.title("Burns-SNooPy Residuals"); plt.xlabel('Redshift'); plt.ylabel('Burns-SNooPy')
    if save:
        plt.savefig('burns-snpy_res.png')
    plt.show()
    return

def solo_plotting(t_o, t_c, flux_o, flux_c, flux_err_o, flux_err_c, err_max, name='Empty', coords=[0, 0], size=(12, 4), save=False, saveLoc=ATLAS_PLOTS_PATH, show_plot=True):
    plt.figure(figsize=size)

    plt.errorbar(t_o, flux_o, yerr=flux_err_o, fmt='o', color='orange')
    plt.errorbar(t_c, flux_c, yerr=flux_err_c, fmt='o', color='cyan')

    plt.title('Light Curve: '+str(name)+'\n'+str(coords[0])+', '+str(coords[1])+'\nMax Error = '+str(err_max))
    plt.xlabel('Time [MJD]')
    plt.ylabel('Flux [uJy]')
    plt.ylim(0)
    plt.savefig(saveLoc+name+'_ATLASplot.png')
    if show_plot:
        plt.show()
    else:
        plt.close()
    return

def combined_plotting(ax, t_o, t_c, flux_o, flux_c, flux_err_o, flux_err_c, name='Empty', coords=[0, 0]):
    ax.errorbar(t_o, flux_o, yerr=flux_err_o, fmt='o', color='orange')
    ax.errorbar(t_c, flux_c, yerr=flux_err_c, fmt='o', color='cyan')

    ax.set_title('Light Curve: '+str(name)+'\n'+str(coords[0])+', '+str(coords[1]))
    ax.set_xlabel('Time [MJD]')
    ax.set_ylabel('Flux [uJy]')
    ax.set_ylim(0)
    return

def plot_ATLAS(xyo, xyc, ax=plt, name='Empty', coords=[0, 0], save=False, saveloc=ATLAS_PLOTS_PATH):
    ax.errorbar(xyo[0], xyo[1], yerr=xyo[2], fmt='o', color='orange')
    ax.errorbar(xyc[0], xyc[1], yerr=xyc[2], fmt='o', color='cyan')

    ax.set_title('Light Curve: '+str(name)+'\n'+str(coords[0])+', '+str(coords[1]))
    ax.set_xlabel('Time [MJD]'); ax.set_ylabel('Flux [uJy]')
    if save:
        plt.savefig(saveloc+name+'_ATLASplot.png')
    return

# Specific functions
def snpy_ASCII_formatting(SNe):
    knownVariables = {}
    skip_header = True
    with open(ROOT_SAVE+'ATLAS_variables.txt', 'r') as f:
        for line in f.readlines():
            if skip_header:
                skip_header = False
                pass
            else:
                line = line[:-1].split(', ')
                name = line[1]
                line.remove(name)
                knownVariables.update({name : line})

    ATLASnames = []
    for SN in SNe:
        ATLASnames.append(str(SN.objname)+'_snpy.txt')
        with open(ATLAS_SNPY_DATA_PATH+str(SN.objname)+'_snpy.txt', 'w') as f:
            # Line 1
            # Ex. SN1981D 0.005871 50.65992 -37.23272
            #     Name    Helio Z  RA        Dec
            try:
                SN.objname = knownVariables[SN.ATLASname][0]
            except KeyError:
                print('No saved data for...', SN.ATLASname)
            f.write(str(SN.objname)+' '+str(SN.z)+' '+str(SN.RA)+' '+str(SN.DEC)+'\n')
            print(str(SN.objname)+' '+str(SN.z)+' '+str(SN.RA)+' '+str(SN.DEC))

            # 'ATri'-filter photometry block
            # Ex. filter O
            #     674.8593      12.94   0.11
            #     Date (JD/MJD) mag     err
            f.write('filter ATri\n')
            for i in range(len(SN.t_o)):
                f.write(str(SN.t_o[i])+'\t'+str(SN.mag_o[i])+'\t'+str(SN.dmag_o[i])+'\n')

            # # 'ATgr'-filter photometry block
            f.write('filter ATgr\n')
            for i in range(len(SN.t_c)):
                f.write(str(SN.t_c[i])+'\t'+str(SN.mag_c[i])+'\t'+str(SN.dmag_c[i])+'\n')
    return ATLASnames

def data_collection(quiet=False):
        if os.path.exists(ROOT_SAVE+'tmp.npz'):
            pickle = np.load(ROOT_SAVE+'tmp.npz', allow_pickle=True)
            data = pickle['data']

        data = requests.post(
            'https://star.pst.qub.ac.uk/sne/atlas4/api/objectlist/',
            headers={'Authorization': f'Token {API_TOKEN}'},
            data={'objectlistid':2}
        ).json()

        np.savez(ROOT_SAVE+'tmp.npz', data=data)

        count = 0
        for d in data:
            if d['observation_status'] is not None and d['observation_status'].startswith('SN Ia') and '91bg' in d['observation_status']:
                count += 1
                if not quiet:
                    print(d['atlas_designation'],d['observation_status'].replace(' ',''),d['ra'],d['dec'])


                ids = d['id']
                base_url = 'https://star.pst.qub.ac.uk/sne/atlas4/lightcurveforced/1161048951013729300/'
                new_url = base_url.replace('1161048951013729300/',str(ids))
                if not quiet:
                    print(new_url)

                idfile = ATLAS_DATA_PATH+'/' + str(ids)+'.txt'
                if os.path.exists(idfile):
                    continue
                urllib.request.urlretrieve(str(new_url), str(idfile))
                if not quiet:
                    print(idfile)

            if count > 300:
                break

def slice_data(path, err_max=100):
    name = path[48:-4]
    data = np.genfromtxt(path, dtype=str, delimiter=',')

    if len(data) == 0:
        print('[!!!] File '+name+' empty...skipping')
        return None

    filters = data[:, 6]
    t = data[:, 8].astype(float)
    flux = data[:, 24]
    dflux = data[:, 25]
    mag = np.char.replace(data[:, 3], '>', '') # Removes greater than symbols
    dmag = data[:, 4]

    # Finds the empty spots of flux and mag and records the element
    mod_empty = np.unique(np.hstack((np.hstack((np.where(flux == 'None')[0], np.where(dflux == 'None')[0])),
                                     np.hstack((np.where(mag == 'None')[0], np.where(dmag == 'None')[0])))))
    filters = np.delete(filters, mod_empty)
    t = np.delete(t, mod_empty).astype(float)
    flux = np.delete(flux, mod_empty).astype(float)
    dflux = np.delete(dflux, mod_empty).astype(float)
    mag = np.delete(mag, mod_empty).astype(float)
    dmag = np.delete(dmag, mod_empty).astype(float)

    # Finds negative fluxes
    mod_positive = np.unique(np.hstack((np.hstack((np.where(flux <= 0)[0], np.where(dflux <= 0)[0])),
                                np.hstack((np.where(mag <= 0)[0], np.where(dmag <= 0)[0])))))
    filters = np.delete(filters, mod_positive)
    t = np.delete(t, mod_positive)
    flux = np.delete(flux, mod_positive)
    dflux = np.delete(dflux, mod_positive)
    mag = np.delete(mag, mod_positive)
    dmag = np.delete(dmag, mod_positive)

    # Find outliers beyond error limit
    mod_err = np.unique(np.hstack(((np.where(abs(dflux) > err_max)[0]), np.where(abs(dmag) > err_max)[0]))) # Negatives fluxes & Error Limit
    filters = np.delete(filters, mod_err)
    t = np.delete(t, mod_err)
    flux = np.delete(flux, mod_err)
    dflux = np.delete(dflux, mod_err)
    mag = np.delete(mag, mod_err)
    dmag = np.delete(dmag, mod_err)

    tempSN = ATLAS_SN(ATLASname=name, RA=np.average(data[:, 1].astype(float)), DEC=np.average(data[:, 2].astype(float)),
                        t_o=t[np.where(filters=='o')[0]], flux_o=flux[np.where(filters=='o')[0]], dflux_o=dflux[np.where(filters=='o')[0]], mag_o=mag[np.where(filters=='o')[0]], dmag_o=dmag[np.where(filters=='o')[0]],
                        t_c=t[np.where(filters=='c')[0]], flux_c=flux[np.where(filters=='c')[0]], dflux_c=dflux[np.where(filters=='c')[0]], mag_c=mag[np.where(filters=='c')[0]], dmag_c=dmag[np.where(filters=='c')[0]])

    return tempSN

def TNS_obj_detection(SNe, t_lim=0.1, saveLoc=ROOT_SAVE+'ATLAS_variables.txt', use_saved=True):
    # Check for presaved data
    savedData = {}
    if os.path.exists(saveLoc) and use_saved:
        data = np.genfromtxt(saveLoc, delimiter=', ', dtype=str, skip_header=1)
        if len(data) > 0:
            for n in range(len(data[:, 0])):
                savedData.update({data[n][1]: [data[n][0], data[n][4]]})

    with open(saveLoc, 'w') as f:
        f.write('objname, ATLASname, RA, DEC, z, tmin, tmax\n') # Header
        for n in range(len(SNe)):
            if SNe[n].ATLASname in savedData:
                obj = {'objname' : savedData[SNe[n].ATLASname][0], 'redshift' : savedData[SNe[n].ATLASname][1]}
                SNe[n].objname = obj['objname']
                SNe[n].z = obj['redshift']
                print("Saved object data found for "+SNe[n].ATLASname+", retrieving...")
            else:
                print("No saved data, sleeping for", t_lim, 'seconds...')
                time.sleep(t_lim)
                obj = TNS_details(ra=SNe[n].RA, dec=SNe[n].DEC)
                SNe[n].objname = 'SN'+obj['objname']
                SNe[n].z = obj['redshift']

            if len(SNe[n].t_o) > 0 and len(SNe[n].t_c) > 0:
                t_max = max([np.max(SNe[n].t_o), np.max(SNe[n].t_c)])
                t_min = min([np.min(SNe[n].t_o), np.min(SNe[n].t_c)])
            elif len(SNe[n].t_o) == 0:
                t_max = np.max(SNe[n].t_c)
                t_min = np.min(SNe[n].t_c)
            elif len(SNe[n].t_c) == 0:
                t_max = np.max(SNe[n].t_o)
                t_min = np.min(SNe[n].t_o)

            print('[ '+str(n+1)+' / '+str(len(SNe))+' ]', SNe[n].ATLASname+':', SNe[n].objname,
                  '| z =', SNe[n].z, '| Tmin:', t_min, 'Tmax:', t_max, '\n')
            f.write(str(SNe[n].objname)+', '+str(SNe[n].ATLASname)+', '+str(SNe[n].RA)+', '+str(SNe[n].DEC)+
                    ', '+str(SNe[n].z)+', '+str(t_min)+', '+str(t_max)+'\n')

    return

# ZTF query functions
def ztf_submit_post(ra_list, dec_list, jds, jde):
    email = 'mekhidw@hawaii.edu' # email you subscribed with.
    userpass = 'wxdk286' # password that was issued to you.

    ra, dec = json.dumps(ra_list), json.dumps(dec_list)
    jdend = json.dumps(jde)
    jdstart = json.dumps(jds)

    payload = {'ra': ra, 'dec': dec, 'jdstart': jdstart, 'jdend': jdend, 'email': email, 'userpass': userpass}

    # fixed IP address/URL where requests are submitted:
    url = 'https://ztfweb.ipac.caltech.edu/cgi-bin/batchfp.py/submit'
    r = requests.post(url, auth=('ztffps', 'dontgocrazy!'), data=payload)

    print(ra, dec)
    print(jdstart)
    print(jdend)
    print("Status_code=",r.status_code)
    return

def ztf_query(targets, jds, jde):
    print("Number of (ra, dec) pairs =", len(targets))

    ralist = []
    declist = []
    i = 0
    for tar in targets:
        raval = float('%.7f'%(float(tar[0])))
        decval = float('%.7f'%(float(tar[1])))
        ralist.append(raval)
        declist.append(decval)
        i = i + 1
        rem = i % 1500 # Limit submission to 1500 sky positions.
        if rem == 0:
            ztf_submit_post(ralist, declist, jds, jde)
            ralist = []
            declist = []
    if len(ralist) > 0:
        ztf_submit_post(ralist, declist, jds, jde)
    return

# Main processing functions
def ATLAS_main_processing(err_max=100, yaxis='flux', t_lim = 0.1, n_iter = 0, plot_mode = 'SOLO', show_plot=True, use_saved=True):
    # [2.1] Aquire Data
    print('[2.1] Retrieving data from...', ATLAS_DATA_PATH)
    files = glob.glob(ATLAS_DATA_PATH+'/*')
    problemChildren = ['1032212120425304400']   # Running list of problematic SNe
                                                # '1032212120425304400' - David believes it might be a shock breakout but ATLAS reports it as SN1a
                                                #
    # [2.2] Validate Data
    print('[2.2] Sorting data...')
    SNe = []
    for n in range(len(files)):
        print('[', n+1, '/', len(files), '] Validating data for...', files[n][48:-4])
        result = slice_data(path=files[n], err_max=err_max)
        if result != None:
            SNe.append(result)
        else:
            problemChildren.append(files[n][48:-4])
        if n_iter != 0 and n+1 >= n_iter:
            break

    # [2.3] Remove problem children
    print('[2.3] Problem Children: ', problemChildren)
    for SN in SNe:
        for probname in problemChildren:
            if SN.ATLASname == probname:
                print('Removing...', SN.ATLASname)

    # [2.4] Save RA, DEC, Redshift, and Object Name
    print('[2.4] Saving parameters to file...')
    print(t_lim, 'second pause between entries...')
    TNS_obj_detection(SNe, t_lim=t_lim, use_saved=use_saved)

    # [2.5] Plot data
    if plot_mode == 'SOLO':
        print("[2.5] Plotting data [indivisual]...")
        with ZipFile(ATLAS_PLOTS_PATH+'/ATLASplots.zip', 'w') as zip_object:
            for SN in SNe:
                solo_plotting(name=SN.ATLASname, coords=[SN.RA, SN.DEC], err_max=err_max,
                              t_o=SN.t_o, flux_o=SN.flux_o, flux_err_o=SN.dflux_o,
                              t_c=SN.t_c, flux_c=SN.flux_c, flux_err_c=SN.dflux_c,
                              save=True, saveLoc=ATLAS_PLOTS_PATH, show_plot=show_plot)
                zip_object.write(ATLAS_PLOTS_PATH+SN.ATLASname+'_ATLASplot.png')
                print('Plot saved to', ATLAS_PLOTS_PATH+SN.ATLASname+'_ATLASplot.png')
    elif plot_mode == 'COMBINED':
        print("[2.5] Plotting data [combined]...")
        fig, axs = plt.subplots(len(SNe), figsize=(12, len(SNe)*3.2))
        fig.tight_layout(pad=5.0)
        for n in range(len(SNe)):
            combined_plotting(ax=axs[n], name=SN.ATLASname, coords=[SN.RA, SN.DEC],
                              t_o=SNe[n].t_o, flux_o=SNe[n].flux_o, flux_err_o=SNe[n].dflux_o,
                              t_c=SN.t_c, flux_c=SN.flux_c, flux_err_c=SN.dflux_c)
        plt.savefig(ATLAS_PLOTS_PATH+'CombinedATLASplots.png')
        print('Plot saved to', ATLAS_PLOTS_PATH+'CombinedATLASplots.png')
        if show_plot:
            plt.show()
        else:
            plt.close()

    return SNe

def ZTF_main_processing(targets, jds, jde):
    print("Number of (ra, dec) pairs =", len(targets))

    ralist = []
    declist = []
    i = 0
    for tar in targets:
        raval = float('%.7f'%(float(tar[0])))
        decval = float('%.7f'%(float(tar[1])))
        ralist.append(raval)
        declist.append(decval)
        i = i + 1
        rem = i % 1500 # Limit submission to 1500 sky positions.
        if rem == 0:
            ztf_submit_post(ralist, declist, jds, jde)
            ralist = []
            declist = []
    if len(ralist) > 0:
        ztf_submit_post(ralist, declist, jds, jde)
    return


In [458]:
""" WORKFLOWS """
def SNooPy_fitting():
    recover_dir()

    # Initialize Data
    KrisciunasNames = np.genfromtxt("/content/HiloCATsSN1991bg/targetLists/91bglike_justnames.txt", dtype=str, delimiter=', ')

    burnsData = np.genfromtxt('/content/HiloCATsSN1991bg/targetLists/burns+25table2ext.txt', dtype=str)
    burnsDistances = {}
    burnsNames = burnsData[:, 0]
    for tar in np.stack((burnsData[:, 0], burnsData[:, 14]), axis=1):
        burnsDistances.update({'SN'+tar[0]: float(tar[1])})

    # Fitting Data
    snpyDistances, snpyRedshifts = SNooPy2_fitting(CPSpath='/content/HiloCATsSN1991bg/data/CSPdata',
                                                   tarNames=KrisciunasNames,
                                                   model='EBV_model2',
                                                   shape='st',
                                                   bands=['B','g','r','i'],
                                                   output=False,
                                                   snpyPlots=True)

    # ATLASnames = glob.glob('/content/ATLASsnpy/*')
    # for n in range(len(ATLASnames)):
    #     ATLASnames[n] = ATLASnames[n][21:-9]
    # snpyDistances, snpyRedshifts, snpyParams = SNooPy2_fitting(CPSpath='/content/ATLASsnpy',
    #                                             tarNames=ATLASnames,
    #                                             model='EBV_model2',
    #                                             shape='st',
    #                                             bands=['ATgr', 'ATri'],
    #                                             output=True,
    #                                             snpyPlots=True)
    # with open('/content/snpy_fit_plots/snpy_fit_params.txt', 'w') as f:
    #     f.write('Object Name, mu, z, st, Tmax, EBVhost\n')
    #     for name in snpyDistances:
    #         f.write(str(name)+', '+str(snpyDistances[name])+', '+str(snpyRedshifts[name])
    #                 +', '+str(snpyParams[name][0])+', '+str(snpyParams[name][1])+', '+str(snpyParams[name][2])+'\n')

    # Plot Distance v. Distance
    plot_DvD(snpyDistances, burnsDistances, size=(8,5), save=True)

    # Plot Residuals
    plot_residuals(snpyDistances, burnsDistances, snpyRedshifts, size=(8,5), save=True)
    return

def ATLAS_process():
    # # [1] Collecting ATLAS data
    # print('[1] Collecting ATLAS data...')
    # data_collection(quiet=False)

    # [2] Slice data, remove outliers, & plot
    print('[2] Processing ATLAS data...')
    SNe = ATLAS_main_processing(err_max=100, t_lim = 10, n_iter = 0, plot_mode = 'SOLO', show_plot=False, use_saved=True)
    print('[2!] ATLAS data processed...', len(SNe), 'SN found and validated.')

    # [3] Write ATLAS data to SNooPy ASCII file
    print('[3] Writing ATLAS data to ASCII file for SNooPy...')
    ATLASnames = snpy_ASCII_formatting(SNe)

    # [4] SNooPy fitting
    print('[4] Fitting ATLAS data with SNooPy...')
    SNe = ATLAS_snpy_fitting(SNe, ATLASnames, show_plot=True, use_saved=True)

    # # [5] SNooPy fit mu v. Redshift
    # plot_SNooPy_mu_vs_z()

    # # [6] Histogram of fit parameters
    # plot_SNooPy_hist(binNum=10)
    return

def ZTF_process():
    # [1] Processing ZTF data
    print('[1] Processing ZTF data...')
    data = np.genfromtxt(ROOT_SAVE+'ATLAS_variables.txt', delimiter=', ', skip_header=1, dtype=str)
    targets = np.stack((data[:, 2], data[:, 4]), axis=1)
    jds = min(data[:, 6])
    jde = max(data[:, 7])
    ZTF_main_processing(targets, jds, jde)
    return


In [459]:
""" MAIN """
if __name__ == '__main__':
    # Runtime tracker
    start = time.time()

    # Recovering vital directories
    recover_dir()

    # Workflow select
    # SNooPy_fitting()
    ATLAS_process()
    # ZTF_process()

    print('|---------------------------|\n Run-time: ', round(time.time()-start, 4), 'seconds\n|---------------------------|')

[2] Processing ATLAS data...
[2.1] Retrieving data from... /content/HiloCATsSN1991bg/saved_data/data/atlas/
[2.2] Sorting data...
[ 1 / 84 ] Validating data for... 1141702841104919500
[ 2 / 84 ] Validating data for... 1015632581035137300
[ 3 / 84 ] Validating data for... 1000438990080559900
[ 4 / 84 ] Validating data for... 1022141900255539900
[ 5 / 84 ] Validating data for... 1121801910205127500
[ 6 / 84 ] Validating data for... 1124332160412157900
[ 7 / 84 ] Validating data for... 1143801871101102900
[ 8 / 84 ] Validating data for... 1115510700075033500
[ 9 / 84 ] Validating data for... 1234414950233145500
[ 10 / 84 ] Validating data for... 1125853461280737700
[ 11 / 84 ] Validating data for... 1140530761154317900
[ 12 / 84 ] Validating data for... 1021103151362854900
[ 13 / 84 ] Validating data for... 1091500861223522500
[ 14 / 84 ] Validating data for... 1030331171274056000
[ 15 / 84 ] Validating data for... 1010733071322429400
[ 16 / 84 ] Validating data for... 1023822740114421300

<ipython-input-457-d88c11512909>:375: UserWarning: genfromtxt: Empty input file: "/content/HiloCATsSN1991bg/saved_data/data/atlas/1140215111094659400.txt"
  data = np.genfromtxt(path, dtype=str, delimiter=',')


[ 30 / 84 ] Validating data for... 1163954751371056500
[ 31 / 84 ] Validating data for... 1062652770243652100
[ 32 / 84 ] Validating data for... 1013624701220300200
[ 33 / 84 ] Validating data for... 1114428341104702000
[!!!] File 1114428341104702000 empty...skipping
[ 34 / 84 ] Validating data for... 1104330091190458400


<ipython-input-457-d88c11512909>:375: UserWarning: genfromtxt: Empty input file: "/content/HiloCATsSN1991bg/saved_data/data/atlas/1114428341104702000.txt"
  data = np.genfromtxt(path, dtype=str, delimiter=',')


[ 35 / 84 ] Validating data for... 1011804611143313100
[ 36 / 84 ] Validating data for... 1122632231312545000
[ 37 / 84 ] Validating data for... 1194757701035644500
[ 38 / 84 ] Validating data for... 1091642021185754300
[ 39 / 84 ] Validating data for... 1132457970241050500
[ 40 / 84 ] Validating data for... 1150925521252818000
[ 41 / 84 ] Validating data for... 1235527530345232200
[ 42 / 84 ] Validating data for... 1062134810271253600
[ 43 / 84 ] Validating data for... 1111155951030803400
[ 44 / 84 ] Validating data for... 1122418881102646700
[ 45 / 84 ] Validating data for... 1124236470404446700
[ 46 / 84 ] Validating data for... 1133245250130248700
[ 47 / 84 ] Validating data for... 1041641640121059200
[ 48 / 84 ] Validating data for... 1114107741244910900
[ 49 / 84 ] Validating data for... 1034109200025054700
[ 50 / 84 ] Validating data for... 1011108430135639500
[ 51 / 84 ] Validating data for... 1040121830544516000
[!!!] File 1040121830544516000 empty...skipping
[ 52 / 84 ] Valid

<ipython-input-457-d88c11512909>:375: UserWarning: genfromtxt: Empty input file: "/content/HiloCATsSN1991bg/saved_data/data/atlas/1040121830544516000.txt"
  data = np.genfromtxt(path, dtype=str, delimiter=',')


 1112142781342129600
[ 59 / 84 ] Validating data for... 1144509771091625100
[ 60 / 84 ] Validating data for... 1040239171251401300
[ 61 / 84 ] Validating data for... 1003753320212821500
[ 62 / 84 ] Validating data for... 1084621190123922600
[ 63 / 84 ] Validating data for... 1081524570065727100
[ 64 / 84 ] Validating data for... 1133923001510301000
[ 65 / 84 ] Validating data for... 1224726940253954200
[ 66 / 84 ] Validating data for... 1094114161260519800
[ 67 / 84 ] Validating data for... 1105545461240141000
[ 68 / 84 ] Validating data for... 1032212120425304400
[ 69 / 84 ] Validating data for... 1063155011353323600
[ 70 / 84 ] Validating data for... 1164320711613542700
[ 71 / 84 ] Validating data for... 1051106310010449800
[ 72 / 84 ] Validating data for... 1050523630115258100
[ 73 / 84 ] Validating data for... 1045902220002631500
[ 74 / 84 ] Validating data for... 1140849041353649800
[ 75 / 84 ] Validating data for... 1191628721415324300
[ 76 / 84 ] Validating data for... 113001461